# Integrating Product Design and Supply Chain Design

## Import Packages

In [124]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from gurobipy import quicksum as qsum
import pandas as pd
from pandas import DataFrame, read_csv
import random
import math

import sys
sys.path.append('../')
import saedfsc

## Parameters

In [125]:
numAssemblyOptions = 2
#numSuppliersPerPart = 2

## Get Part Options

In [126]:
partOptions = saedfsc.getPartOptionsWithSuppliers()

## Suppliers

In [127]:
saedfsc.suppliers

,Name,Location,Rating,SetUpCost,Cost Per Unit,Time Per Unit
0,Supplier1,Location3,3.536724,4665,9,8
1,Supplier2,Location2,3.202017,4095,5,3
2,Supplier3,Location4,4.723231,1178,8,9
3,Supplier4,Location1,2.719848,2503,4,2
4,Supplier5,Location3,4.765031,4383,8,7
5,Supplier6,Location3,4.470437,3035,2,4
6,Supplier7,Location3,2.462482,3477,1,8


In [128]:
latex_table = saedfsc.suppliers.to_latex(index=False)
print(latex_table)

\begin{tabular}{llrrrr}
\toprule
Name & Location & Rating & SetUpCost & Cost Per Unit & Time Per Unit \\
\midrule
Supplier1 & Location3 & 3.536724 & 4665 & 9 & 8 \\
Supplier2 & Location2 & 3.202017 & 4095 & 5 & 3 \\
Supplier3 & Location4 & 4.723231 & 1178 & 8 & 9 \\
Supplier4 & Location1 & 2.719848 & 2503 & 4 & 2 \\
Supplier5 & Location3 & 4.765031 & 4383 & 8 & 7 \\
Supplier6 & Location3 & 4.470437 & 3035 & 2 & 4 \\
Supplier7 & Location3 & 2.462482 & 3477 & 1 & 8 \\
\bottomrule
\end{tabular}



## Create Components and Assemblies

In [129]:

components = []
componentsForSubsystems = {}
subsystemsForComponents = {}
for subsystem in partOptions.keys():
    if subsystem == 'wings':
        toAdd = ['rear wing', 'front wing', 'side wing']
        for c in toAdd:
            components.append(c)
            subsystemsForComponents[c] = subsystem
        componentsForSubsystems[subsystem] = toAdd
    elif subsystem == 'tires':
        toAdd = ['front tire', 'rear tire']
        for c in toAdd:
            components.append(c)
            subsystemsForComponents[c] = subsystem
        componentsForSubsystems[subsystem] = toAdd
    elif subsystem == 'suspension':
        #toAdd = ['front suspension', 'rear suspension']
        toAdd = ['suspension']
        for c in toAdd:
            components.append(c)
            subsystemsForComponents[c] = subsystem
        componentsForSubsystems[subsystem] = toAdd
    else:
        components.append(subsystem)
        componentsForSubsystems[subsystem] = [subsystem]
        subsystemsForComponents[subsystem] = subsystem
assembliesStructure = {'midbody' : ['engine', 'cabin', 'side wing'], 
                       'front' : ['front wing', 'front tire', 'impactattenuator', 'suspension'], 
                       'rear' : ['rear wing', 'brakes', 'rear tire'],}
assemblyNodes = list(assembliesStructure.keys())
finalNodes = ['FINAL']
allNodes = components + assemblyNodes + finalNodes
nonComponentNodes = assemblyNodes + finalNodes

## Define function to create nominalPartPrices dictionary

In [130]:
def make_nominal_part_prices(broad_part_names, part_options):
    nominalPartPrices = {}
    for broad_part in broad_part_names:
        broad_part_price_dict = {}
        for specific_parts in range(len(partOptions[broad_part])):
            if broad_part == 'wings':
                
                broad_part_price_dict[part_options['wings'].index[specific_parts]] = round(part_options['wings'].iloc[specific_parts]['length'] * 
                                                                                     part_options['wings'].iloc[specific_parts]['width'] * 
                                                                                     part_options['wings'].iloc[specific_parts]['height'] * 
                                                                                     part_options['wings'].iloc[specific_parts]['q'] * 
                                                                                     part_options['wings'].iloc[specific_parts]['cost_per_kilogram'], 2)
            
            elif broad_part == 'tires':
                
                broad_part_price_dict[part_options['tires'].index[specific_parts]] = round(part_options['tires'].iloc[specific_parts]['cost'] * 
                                                                                     (1 + (part_options['tires'].iloc[specific_parts]['pressure'] - 0.758) / 2), 2)
                                            
                
            elif broad_part == 'engine':
                
                broad_part_price_dict[part_options['engine'].index[specific_parts]] = round(part_options['engine'].iloc[specific_parts]['Cost'], 2)
                
            elif broad_part == 'cabin':
                
                broad_part_price_dict[part_options['cabin'].index[specific_parts]] = round(2 * 
                                                                                     part_options['cabin'].iloc[specific_parts]['thickness'] * 
                                                                                     (part_options['cabin'].iloc[specific_parts]['length'] * 
                                                                                      part_options['cabin'].iloc[specific_parts]['width'] + 
                                                                                      part_options['cabin'].iloc[specific_parts]['length'] * 
                                                                                      part_options['cabin'].iloc[specific_parts]['height'] + 
                                                                                      part_options['cabin'].iloc[specific_parts]['width'] * 
                                                                                      part_options['cabin'].iloc[specific_parts]['height']  
                                                                                     ) *
                                                                                     part_options['cabin'].iloc[specific_parts]['q'] * 
                                                                                     part_options['cabin'].iloc[specific_parts]['cost_per_kilogram'], 2)
                
            elif broad_part == 'brakes':
                
                broad_part_price_dict[part_options['brakes'].index[specific_parts]] = round(part_options['brakes'].iloc[specific_parts]['lbrk'] * 
                                                                                      part_options['brakes'].iloc[specific_parts]['wbrk'] * 
                                                                                      part_options['brakes'].iloc[specific_parts]['hbrk'] * 
                                                                                      part_options['brakes'].iloc[specific_parts]['qbrk'] * 
                                                                                      1000 * 
                                                                                      25, 2)
                                            
                
            elif broad_part == 'impactattenuator':
                
                broad_part_price_dict[part_options['impactattenuator'].index[specific_parts]] = round(part_options['impactattenuator'].iloc[specific_parts]['length'] * 
                                                                                                part_options['impactattenuator'].iloc[specific_parts]['width'] * 
                                                                                                part_options['impactattenuator'].iloc[specific_parts]['height'] * 
                                                                                                part_options['impactattenuator'].iloc[specific_parts]['q'] * 
                                                                                                part_options['impactattenuator'].iloc[specific_parts]['cost_per_kilogram'], 2)
                
                
            elif broad_part == 'suspension':
                
                broad_part_price_dict[part_options['suspension'].index[specific_parts]] = 0 # Assuming suspension has fixed cost and is "tuned" (Comment from Dr. McComb's code)
            
            
        nominalPartPrices[broad_part] = broad_part_price_dict
        
    return nominalPartPrices

In [131]:
# Example use of function make_nominal_part_prices

nominalPartPrices = make_nominal_part_prices(partOptions.keys(), partOptions)

## Display Graph

In [132]:
G = nx.DiGraph()

numNodes = len(components) + len(assemblyNodes) + len(finalNodes)

np.random.seed(0)
stageCostsList = np.random.randint(1, 10, numNodes)
processTimesList = np.random.randint(1, 10, numNodes)
maxServiceTimeOutList = 200*np.ones(numNodes)
maxServiceTimeOutList[numNodes-1] = 0

for n in allNodes:
    time = np.random.randint(1, 10)
    G.add_node(n, process_time=time, 
               max_service_time_out=2*time,
               stage_cost=np.random.randint(1, 10))

for n in assemblyNodes:
    for component in assembliesStructure[n]:
        G.add_edge(component, n)

for n in assemblyNodes:
    for n2 in finalNodes:
        G.add_edge(n, n2)

maxServiceTimeOut = nx.get_node_attributes(G, 'max_service_time_out')

# nx.draw(G, with_labels=True, node_size=2000)
# plt.show()

## Options for Each Node (Component or Assembly)

In [133]:
optionsInfo = {}
optionsForSubSysAndPart = {}
numOptionsForNode = {}
pI = saedfsc.procurementInfo
for n in G.nodes():
    if n in components:
        k = 0
        subsys = subsystemsForComponents[n]
        for p in partOptions[subsys].index.values:
            optionsForSubSysAndPart[subsys,p] = []
            for supplier in partOptions[subsys].loc[p, 'Suppliers'].tolist():
                cost_per_unit = np.random.poisson(lam=nominalPartPrices[subsys][p])
                for proc in range(len(pI)):
                    if pI.loc[proc]['Supplier'] == supplier and pI.loc[proc]['Broader Part'] == subsys and pI.loc[proc]['Specific Part: Row Index'] == p:
                        time_per_unit = pI.loc[proc]['Lead Times']
                optionsInfo[n,k] = {'cost' : cost_per_unit, 'time' : time_per_unit}
                optionsForSubSysAndPart[subsys,p].append(k)
                k += 1
        numOptionsForNode[n] = k
    else:
        numOptionsForNode[n] = numAssemblyOptions
        for k in range(numAssemblyOptions):
            optionsInfo[n,k] = {'cost' : np.random.randint(1, 10), 'time' : np.random.randint(1, 10)}

flattened_optionsInfo = {(i,k): values for (i,k), values in optionsInfo.items()}

df = pd.DataFrame.from_dict(flattened_optionsInfo, orient='index')

df.reset_index(inplace=True)
df.columns = ['Node', 'Option', 'Cost', 'Time']
#print(df.to_string())

## Customers

In [134]:
customersDF = saedfsc.getCustomersDF()
customerTypes = customersDF['Name'].to_list()
meanDemandForCustomerType = {c : np.random.randint(1, 20) for c in customerTypes}
stDevDemandForCustomerType = {c : 3 for c in customerTypes}
meanDemands = dict(zip(customersDF['Name'], customersDF['Mean Demand']))
stdDevDemands = dict(zip(customersDF['Name'], customersDF['Std Dev Demand']))
cPriceFocus = dict(zip(customersDF['Name'], customersDF['PriceFocus']))
#name_weights_partworth_dict = customersDF.set_index('Name')['PartworthUtilityWeights'].to_dict()
name_weights_perf_dict = customersDF.set_index('Name')['PerformanceUtilityWeights'].to_dict()
#cWtsPartworth = {c : np.fromstring(name_weights_partworth_dict[c].strip('[]'), sep=',') for c in name_weights_partworth_dict}
cWtsPerf = {c : np.fromstring(name_weights_perf_dict[c].strip('[]'), sep=',') for c in name_weights_perf_dict}
customersDF

,Name,Quantity,Mean Demand,Std Dev Demand,PriceFocus,PartworthUtilityWeights,PerformanceUtilityWeights
0,CustomerType1,550,23,4,0.185002,"[0.05502635,0.15548226,0.1138455 ,0.15706312,0...","[0.10403705,0.17744796,0.11303432,0.01412367,0..."
1,CustomerType2,32,19,2,0.721966,"[0.05439101,0.0698292 ,0.1680139 ,0.07585734,0...","[0.19323785,0.15236526,0.06466306,0.07034756,0..."
2,CustomerType3,804,21,4,0.489219,"[0.09282521,0.08616018,0.1542277 ,0.13957106,0...","[0.13414659,0.13715431,0.03786418,0.11151464,0..."


In [135]:
latex_table = customersDF.to_latex(index=False)
print(latex_table)

\begin{tabular}{lrrrrll}
\toprule
Name & Quantity & Mean Demand & Std Dev Demand & PriceFocus & PartworthUtilityWeights & PerformanceUtilityWeights \\
\midrule
CustomerType1 & 550 & 23 & 4 & 0.185002 & [0.05502635,0.15548226,0.1138455 ,0.15706312,0.06878674,0.05107079,
 0.13021309,0.10490887,0.1335425 ,0.0300608 ] & [0.10403705,0.17744796,0.11303432,0.01412367,0.09074952,0.06196034,
 0.04152583,0.07723446,0.08962796,0.14334552,0.08691335] \\
CustomerType2 & 32 & 19 & 2 & 0.721966 & [0.05439101,0.0698292 ,0.1680139 ,0.07585734,0.1660349 ,0.0230838 ,
 0.05777839,0.05211967,0.1209466 ,0.21194521] & [0.19323785,0.15236526,0.06466306,0.07034756,0.00089544,0.00064008,
 0.12437685,0.1397965 ,0.07107653,0.14266048,0.03994039] \\
CustomerType3 & 804 & 21 & 4 & 0.489219 & [0.09282521,0.08616018,0.1542277 ,0.13957106,0.05241416,0.07873523,
 0.06411288,0.18716082,0.03972689,0.10506587] & [0.13414659,0.13715431,0.03786418,0.11151464,0.12350755,0.12949936,
 0.13148834,0.01641187,0.05542951,0.0642533

## Objectives (Performance Metrics)

In [136]:
objectives = ['mass', 'center of gravity', 'drag', 'downforce', 'acceleration', 
              'crash force', 'attenuator volume', 'cornering velocity', 
              'braking distance', 'suspension acceleration', 'pitch moment']
objIndices = range(len(objectives))

## Define Parameters and Sets

In [137]:
meanDemand = 5
stdDevDemand = 3
holdingCostRate = 1
safetyFactor = 2
betaMult = 5
#numComponentsRequired = len(components) - 1
price = 100000
maxProductPrice = 200000

holdCostCoef = holdingCostRate*safetyFactor

## Parameters for COTSCar Model

In [138]:
random.seed(10) # Since uniform distribution used in center of gravity constraints

v_car = 26.8 #m/s
omega_e = 3600 #rpm
rho_air = 1.225 #kg/m^3
r_track = 9 #m
P_brk = 1 * (10 ** 7) #Pa
g = 9.81 #m/s^2
y_parameter = 0.05 #m
y_dot_parameter = 0.025 #m/s
c_brk = 0.37 # Coefficient of Brake Friction
l_f = 0.5 # Pitch Moment Parameter
C_lat = 1.6 # Cornering Velocity parameter in Dr. McComb code

# Car performance objectives minimum values
objectives_min = {'mass': 95.4413,
                  'center of gravity': 0.1159,
                  'drag': 4.8283,
                  'downforce': 0.006299,
                  'acceleration': 0.2,
                  'crash force': 1636425.0,
                  'attenuator volume': 0.004049,
                  'cornering velocity': 0.01908,
                  'braking distance': 5.9088,
                  'suspension acceleration': 9.81,
                  'pitch moment': 0.03386}

# Car performance objectives maximum values
objectives_max = {'mass': 5593.26,
                  'center of gravity': 0.9996,
                  'drag': 633.95,
                  'downforce': 3211.16,
                  'acceleration':  4.35,
                  'crash force': 672530217,
                  'attenuator volume': 0.1579,
                  'cornering velocity': 15.0652,
                  'braking distance': 553.92,
                  'suspension acceleration': 18.79,
                  'pitch moment': 9850.87}

# Weights of each car performance term in objective function (Scenario 1)
performance_weights = [0.14, 0.01, 0.20, 0.30, 0.10, 0.01, 0.01, 0.10, 0.10, 0.02, 0.01] 

## Convenience Methods

This method is a placeholder until we model competitors using the competitors DataFrame.

In [139]:
numCompetitors = 2
minUtil = 0
maxUtil = 1
totalUtiltityForCompetitors = {c : sum(np.random.uniform(minUtil, maxUtil, numCompetitors)) 
                               for c in customerTypes} # 

def getTotalUtilityForCompetitors(customerType : int):
    return totalUtiltityForCompetitors[customerType]

## Create Model and Add Variables

In [140]:
m = gp.Model("PD-SC Optimization")

Get bounds of variables.

In [141]:
maxStdDevDemand = np.sqrt(sum(stdDevDemands.values()))
maxMeanDemand = sum(meanDemands.values())
maxNetTime = sum(processTimesList)

# improve these bounds later
maxStageTime = 1000 
maxStageCost = 100000
maxServiceIn = 1000
maxCumCost = 100000

In [142]:

stdDevDemandVar = m.addVar(name="stdDevDemand", ub = maxStdDevDemand)
meanDemandVar = m.addVar(name="meanDemand", ub = maxMeanDemand)
marketShare = m.addVars(customerTypes, ub = 1, name="marketShare")
customerUtil = m.addVars(customerTypes, ub = 1, name="customerUtil")
componentSelect = m.addVars(components, vtype=GRB.BINARY, 
                            name="componentSelect")
arcSelect = m.addVars(G.edges(), vtype=GRB.BINARY, name="arcSelect")
stageTime = m.addVars(G.nodes(), vtype=GRB.INTEGER, ub = maxStageTime,
                    name="stageTime")
stageCost = m.addVars(G.nodes(), vtype=GRB.INTEGER, ub = maxStageCost,
                    name="stageCost")
sIn = m.addVars(G.nodes(), vtype=GRB.INTEGER, ub = maxServiceIn,
                    name="sIn")
sOut = m.addVars(G.nodes(), ub = maxServiceTimeOut, vtype=GRB.INTEGER, 
                 name="sOut")
netTime = m.addVars(G.nodes(), ub = maxNetTime, vtype=GRB.INTEGER, 
                    name="netTime")
sqrtNetTime = m.addVars(G.nodes(), ub = np.sqrt(maxNetTime), 
                        name="sqRootnetTime")
cumCostVar = m.addVars(G.nodes(), ub = maxCumCost, name="cumCost")

stdDevDemandTimesCumCostVar = m.addVars(G.nodes(), 
                                        name="stdDevDemandTimesCumCost")
meanDemandTimesStageTimeVar = m.addVars(G.nodes(), 
                                        name="meanDemandTimesStageTime");

This variable holds the value for each of the performance objectives.

In [143]:
z = m.addVars(objectives, lb = objectives_min, ub = objectives_max, 
                    vtype = GRB.CONTINUOUS, 
                    name="objValRaw") # Raw car performance values

z_norm = m.addVars(objectives, ub = 1, 
                    name="objValNorm") # Normalized [0,1] car performance values 

This variable selections which option to use for each node (components, assemblies, and the final node). An option determines both the part as well as a supplier

In [144]:
optionSelect = m.addVars(optionsInfo.keys(), vtype=GRB.BINARY, 
                            name="optionSelect")

This variable determines which part is used for each component.

In [145]:
x = {} # part selection variables (parts to use for each component)
for subsystem in partOptions.keys():
    for c in componentsForSubsystems[subsystem]:
        x[c] = m.addVars(partOptions[subsystem].index.values, vtype=GRB.BINARY, name="x_{0}".format(c))

## Add Variables for Car Design

In [146]:
u_e_mTotal = m.addVars(partOptions['engine'].index.values, vtype = GRB.CONTINUOUS, name = "u_e_mTotal") # x_e * mTotal Equivalency Variable
u_rootMass = m.addVar(1, vtype = GRB.CONTINUOUS, name = 'u_rootMass') # square root of total mass equivalence variable
u_rt_brkDistance = m.addVars(partOptions['tires'].index.values, vtype = GRB.CONTINUOUS, name = 'u_rt_downforce') # Rear Tire and Downforce Equivalency Variable
u_rt_pressure_inverse = m.addVars(partOptions['tires'].index.values, vtype = GRB.CONTINUOUS, name = 'u_rt_pressure_inverse') # Inverse of rear tire pressure for C calculation
u_rw_c = m.addVars(partOptions['wings'].index.values, partOptions['cabin'].index.values, vtype = GRB.BINARY, name = 'u_rw_c') # rear wing and cabin equivalence variable
u_fw_c = m.addVars(partOptions['wings'].index.values, partOptions['cabin'].index.values, vtype = GRB.BINARY, name = 'u_fw_c') # front wing and cabin equivalence variable
u_sw_c = m.addVars(partOptions['wings'].index.values, partOptions['cabin'].index.values, vtype = GRB.BINARY, name = 'u_sw_c') # side wing and cabin equivalence variable

## Methods to Help in Creating Model

In [147]:
def outFlowSum(i):
    return qsum(arcSelect[i,j] for j in G.successors(i))

def inFlowSum(i): 
    return qsum(arcSelect[j,i] for j in G.predecessors(i))

def getUtilityForProduct(customer : int):
    pricePerf = (maxProductPrice - price) / maxProductPrice # Normalized price (0 is best, 1 is worst)
    product_utility = qsum(cWtsPerf[customer][o]*z_norm[objectives[o]] for o in objIndices)
    return (1-cPriceFocus[customer])*product_utility + cPriceFocus[customer]*pricePerf

## Add Constraints and Objective

In [148]:
m.addConstrs((qsum(optionSelect[c,k] for k in optionsForSubSysAndPart[subsystem, p]) == x[c][p] 
                   for subsystem in partOptions.keys()
                   for c in componentsForSubsystems[subsystem]
                   for p in partOptions[subsystem].index.values), 
             "options-for-part")
m.addConstrs((stageCost[i] == qsum(optionsInfo[i,k]['cost']*optionSelect[i,k] 
                                   for k in range(numOptionsForNode[i])) 
              for i in G.nodes()), 
              name="StageCostCalc")
m.addConstrs((stageTime[i] == qsum(optionsInfo[i,k]['time']*optionSelect[i,k] 
                                   for k in range(numOptionsForNode[i])) 
              for i in G.nodes()), 
              name="StageTimeCalc")
m.addConstrs((cumCostVar[j] == stageCost[j] + qsum(cumCostVar[i]*arcSelect[i,j] 
                                                    for i in G.predecessors(j)) 
                                                    for j in G.nodes()), 
             name="calcCumCost")
m.addConstrs((sIn[j] >= sOut[i]*arcSelect[i,j] for i,j in G.edges()), 
             name="sInGreater")
m.addConstrs((optionSelect.sum(i,'*') == componentSelect[i] for i in components), 
             "select1-component")
m.addConstrs((optionSelect.sum(i,'*') == 1 for i in nonComponentNodes), 
             "select1-other")
m.addConstrs((inFlowSum(i) == G.in_degree(i)*outFlowSum(i) for i in assemblyNodes), 
             name="netFlow")
m.addConstrs((qsum(arcSelect[i,j] for j in G.successors(i)) == componentSelect[i] 
              for i in components), 
             name="sumOutFlow")
m.addConstrs((arcSelect[i,j] <= componentSelect[i] 
              for i,j in G.edges() if i in components), 
              "useArcIfNode")
m.addConstrs((sIn[i] == 0 for i in components), 
             name="procureNodesInTime")
m.addConstrs((netTime[i] == sIn[i] + stageTime[i] - sOut[i] 
              for i in G.nodes()), 
              name="NetTimeCalc")
m.addConstrs((netTime[i] == sqrtNetTime[i]*sqrtNetTime[i] 
              for i in G.nodes()), 
             name="SqRootNetTimeCalc")
m.addConstrs((stdDevDemandTimesCumCostVar[i] == stdDevDemandVar*cumCostVar[i] 
              for i in G.nodes()), 
             name="stdDevDemandTimesCumCostVar")
m.addConstrs((meanDemandTimesStageTimeVar[i] == meanDemandVar*stageTime[i] 
              for i in G.nodes()), 
             name="meanDemandTimesStageTimeVar")
m.addConstr(meanDemandVar == qsum(meanDemandForCustomerType[c]*marketShare[c] for c in customerTypes), 
            name="meanDemandVar")
m.addConstr(stdDevDemandVar**2 >= qsum(stDevDemandForCustomerType[c]**2*marketShare[c]**2 
                                       for c in customerTypes), 
            name="stdDevDemandVar")
m.addConstrs((customerUtil[c]**2 <= getUtilityForProduct(c) 
              for c in customerTypes), 
             name="customerUtilCalc") #sqrt utility function (diminishing returns)
m.addConstrs((customerUtil[c] == getTotalUtilityForCompetitors(c)*marketShare[c] + customerUtil[c]*marketShare[c] 
              for c in customerTypes), # linearized form of logit function
             name="marketShareCalc")

# placeholder constraints to make example work
componentsRequired = components
m.addConstrs((componentSelect[i] == 1 for i in componentsRequired), 
             name="componentsRequired")
#m.addConstr(qsum(componentSelect[i] for i in components) == 11, # all components must be used
#             name="componentsRequired")

m.setObjective(price*meanDemandVar 
               - qsum(holdCostCoef*stdDevDemandTimesCumCostVar[i]*sqrtNetTime[i] 
                    + holdingCostRate*(cumCostVar[i] - stageCost[i]/2)*meanDemandTimesStageTimeVar[i]
                    + betaMult*stageCost[i]*meanDemandVar
                                            for i in G.nodes()), 
                                                GRB.MAXIMIZE); # maximize expected profit

In [149]:
# Mass Constraints

rw_mass = gp.quicksum(partOptions['wings'].loc[part]['length'] * 
                      partOptions['wings'].loc[part]['width'] * 
                      partOptions['wings'].loc[part]['height'] * 
                      partOptions['wings'].loc[part]['q'] *
                      x['rear wing'][part] for part in partOptions['wings'].index.values)

fw_mass = gp.quicksum(partOptions['wings'].loc[part]['length'] * 
                      partOptions['wings'].loc[part]['width'] * 
                      partOptions['wings'].loc[part]['height'] * 
                      partOptions['wings'].loc[part]['q'] *
                      x['front wing'][part] for part in partOptions['wings'].index.values)

sw_mass = gp.quicksum(partOptions['wings'].loc[part]['length'] * 
                      partOptions['wings'].loc[part]['width'] * 
                      partOptions['wings'].loc[part]['height'] * 
                      partOptions['wings'].loc[part]['q'] *
                      x['side wing'][part] for part in partOptions['wings'].index.values)

ft_mass = gp.quicksum(partOptions['tires'].loc[part]['mass'] * 
                      x['front tire'][part] for part in partOptions['tires'].index.values)

rt_mass = gp.quicksum(partOptions['tires'].loc[part]['mass'] * 
                      x['rear tire'][part] for part in partOptions['tires'].index.values)

c_mass = 2 * gp.quicksum(partOptions['cabin'].loc[part]['thickness'] * 
                         (partOptions['cabin'].loc[part]['length'] * 
                          partOptions['cabin'].loc[part]['width'] + 
                          partOptions['cabin'].loc[part]['length'] * 
                          partOptions['cabin'].loc[part]['height'] + 
                          partOptions['cabin'].loc[part]['width'] * 
                          partOptions['cabin'].loc[part]['height']) * 
                         partOptions['cabin'].loc[part]['q'] * 
                         x['cabin'][part] for part in partOptions['cabin'].index.values)

e_mass = gp.quicksum(partOptions['engine'].loc[part]['Mass'] * 
                     x['engine'][part] for part in partOptions['engine'].index.values)

ia_mass = gp.quicksum(partOptions['impactattenuator'].loc[part]['length'] * 
                      partOptions['impactattenuator'].loc[part]['width'] * 
                      partOptions['impactattenuator'].loc[part]['height'] * 
                      partOptions['impactattenuator'].loc[part]['q'] * 
                      x['impactattenuator'][part] for part in partOptions['impactattenuator'].index.values)

brk_mass = gp.quicksum(partOptions['brakes'].loc[part]['lbrk'] * 
                       partOptions['brakes'].loc[part]['wbrk'] * 
                       partOptions['brakes'].loc[part]['hbrk'] * 
                       partOptions['brakes'].loc[part]['qbrk'] * 
                       1000 * 
                       x['brakes'][part] for part in partOptions['brakes'].index.values)
                      
rsp_mass = gp.quicksum(partOptions['suspension'].loc[part]['mrsp'] * 
                       x['suspension'][part] for part in partOptions['suspension'].index.values)

fsp_mass = gp.quicksum(partOptions['suspension'].loc[part]['mfsp'] * 
                       x['suspension'][part] for part in partOptions['suspension'].index.values)
                      
total_mass = (rw_mass + fw_mass + (2 * sw_mass) + (2 * ft_mass) + (2 * rt_mass) +
             c_mass + e_mass + ia_mass + (4 * brk_mass) + (2 * rsp_mass) + (2 * fsp_mass))
    
m.addConstr(z['mass'] == total_mass)
m.update()

In [150]:
# Center of Gravity Constraints

rw_position_y = gp.quicksum(np.random.uniform(0.5 + (partOptions['wings'].loc[part]['height'] / 2), 
                                              1.2 - (partOptions['wings'].loc[part]['height'] / 2)) *
                                              x['rear wing'][part] for part in partOptions['wings'].index.values)

fw_position_y = gp.quicksum(np.random.uniform(0.03 + (partOptions['wings'].loc[part]['height'] / 2), 
                                              0.25 - (partOptions['wings'].loc[part]['height'] / 2)) *
                                              x['front wing'][part] for part in partOptions['wings'].index.values)

sw_position_y = gp.quicksum(np.random.uniform(0.03 + (partOptions['wings'].loc[part]['height'] / 2), 
                                              0.25 - (partOptions['wings'].loc[part]['height'] / 2)) * 
                                              x['side wing'][part] for part in partOptions['wings'].index.values)

rt_position_y = gp.quicksum(partOptions['tires'].loc[part]['radius'] * 
                            x['rear tire'][part] for part in partOptions['tires'].index.values)

ft_position_y = gp.quicksum(partOptions['tires'].loc[part]['radius'] * 
                            x['front tire'][part] for part in partOptions['tires'].index.values)

c_position_y = gp.quicksum(np.random.uniform(0.03 + (partOptions['cabin'].loc[part]['height'] / 2), 
                                             1.2 - (partOptions['cabin'].loc[part]['height'] / 2)) * 
                                             x['cabin'][part] for part in partOptions['cabin'].index.values)

e_position_y = gp.quicksum(np.random.uniform(0.03 + (partOptions['engine'].loc[part]['Height'] / 2), 
                                             0.50 - (partOptions['engine'].loc[part]['Height'] / 2)) * 
                                             x['engine'][part] for part in partOptions['engine'].index.values)

ia_position_y = gp.quicksum(np.random.uniform(0.03 + (partOptions['impactattenuator'].loc[part]['height'] / 2), 
                                              1.20 - (partOptions['impactattenuator'].loc[part]['height'] / 2)) * 
                                              x['impactattenuator'][part] for part in partOptions['impactattenuator'].index.values)

brk_position_y = gp.quicksum(partOptions['tires'].loc[part]['radius'] * 
                             x['front tire'][part] for part in partOptions['tires'].index.values)

rsp_position_y = gp.quicksum(np.random.uniform(partOptions['tires'].loc[part]['radius'], 
                                               2 * partOptions['tires'].loc[part]['radius']) * 
                                               x['rear tire'][part] for part in partOptions['tires'].index.values)

fsp_position_y = gp.quicksum(np.random.uniform(partOptions['tires'].loc[part]['radius'], 
                                               2 * partOptions['tires'].loc[part]['radius']) * 
                                               x['front tire'][part] for part in partOptions['tires'].index.values)

m.addConstr(z['center of gravity'] * z['mass'] == (rw_mass * rw_position_y) + 
                                                  (fw_mass * fw_position_y) + 
                                                  (2 * sw_mass * sw_position_y) + 
                                                  (2 * ft_mass * ft_position_y) + 
                                                  (2 * rt_mass * rt_position_y) + 
                                                  (c_mass * c_position_y) + 
                                                  (e_mass * e_position_y) + 
                                                  (ia_mass * ia_position_y) + 
                                                  (4 * brk_mass * brk_position_y) + 
                                                  (2 * rsp_mass * rsp_position_y) + 
                                                  (2 * fsp_mass * fsp_position_y))
m.update()

In [151]:
# Drag Constraints

rw_drag = gp.quicksum(((2 * 
                       (partOptions['wings'].loc[part]['width'] ** 2) * 
                       (partOptions['wings'].loc[part]['angle of attack'] ** 2) * 
                       (v_car ** 2) * 
                       partOptions['wings'].loc[part]['height'] * 
                       rho_air * 
                       math.pi * 
                       math.cos(partOptions['wings'].loc[part]['angle of attack'])) / 
                      (partOptions['wings'].loc[part]['length'] * 
                       (((partOptions['wings'].loc[part]['width'] * 
                          math.cos(partOptions['wings'].loc[part]['angle of attack']) / 
                          partOptions['wings'].loc[part]['length']) + 2) ** 2))) * 
                      x['rear wing'][part] for part in partOptions['wings'].index.values)

fw_drag = gp.quicksum(((2 * 
                       (partOptions['wings'].loc[part]['width'] ** 2) * 
                       (partOptions['wings'].loc[part]['angle of attack'] ** 2) * 
                       (v_car ** 2) * 
                       partOptions['wings'].loc[part]['height'] * 
                       rho_air * 
                       math.pi * 
                       math.cos(partOptions['wings'].loc[part]['angle of attack'])) / 
                      (partOptions['wings'].loc[part]['length'] * 
                       (((partOptions['wings'].loc[part]['width'] * 
                          math.cos(partOptions['wings'].loc[part]['angle of attack']) / 
                          partOptions['wings'].loc[part]['length']) + 2) ** 2))) * 
                      x['front wing'][part] for part in partOptions['wings'].index.values)

sw_drag = gp.quicksum(((2 * 
                       (partOptions['wings'].loc[part]['width'] ** 2) * 
                       (partOptions['wings'].loc[part]['angle of attack'] ** 2) * 
                       (v_car ** 2) * 
                       partOptions['wings'].loc[part]['height'] * 
                       rho_air * 
                       math.pi * 
                       math.cos(partOptions['wings'].loc[part]['angle of attack'])) / 
                      (partOptions['wings'].loc[part]['length'] * 
                       (((partOptions['wings'].loc[part]['width'] * 
                          math.cos(partOptions['wings'].loc[part]['angle of attack']) / 
                          partOptions['wings'].loc[part]['length']) + 2) ** 2))) * 
                      x['side wing'][part] for part in partOptions['wings'].index.values)

c_drag = gp.quicksum(0.02 * 
                     rho_air * 
                     (v_car ** 2) * 
                     partOptions['cabin'].loc[part]['width'] * 
                     partOptions['cabin'].loc[part]['height'] * 
                     x['cabin'][part] for part in partOptions['cabin'].index.values)

total_drag = rw_drag + fw_drag + (2 * sw_drag) + c_drag

m.addConstr(z['drag'] == total_drag)
m.update()

In [152]:
# Downforce Constraints

rw_downforce = gp.quicksum((((partOptions['wings'].loc[part]['angle of attack'] ** 2) * 
                             (partOptions['wings'].loc[part]['width'] ** 2) * 
                             (v_car ** 2) * 
                             partOptions['wings'].loc[part]['height'] * 
                             rho_air * 
                             math.pi * 
                             math.cos(partOptions['wings'].loc[part]['angle of attack'])) / 
                            ((partOptions['wings'].loc[part]['width'] * 
                              math.cos(partOptions['wings'].loc[part]['angle of attack'])) + 
                             (2 * partOptions['wings'].loc[part]['length']))) * 
                            x['rear wing'][part] for part in partOptions['wings'].index.values)

fw_downforce = gp.quicksum((((partOptions['wings'].loc[part]['angle of attack'] ** 2) * 
                             (partOptions['wings'].loc[part]['width'] ** 2) * 
                             (v_car ** 2) * 
                             partOptions['wings'].loc[part]['height'] * 
                             rho_air * 
                             math.pi * 
                             math.cos(partOptions['wings'].loc[part]['angle of attack'])) / 
                            ((partOptions['wings'].loc[part]['width'] * 
                              math.cos(partOptions['wings'].loc[part]['angle of attack'])) + 
                             (2 * partOptions['wings'].loc[part]['length']))) * 
                            x['front wing'][part] for part in partOptions['wings'].index.values)

sw_downforce = gp.quicksum((((partOptions['wings'].loc[part]['angle of attack'] ** 2) * 
                             (partOptions['wings'].loc[part]['width'] ** 2) * 
                             (v_car ** 2) * 
                             partOptions['wings'].loc[part]['height'] * 
                             rho_air * 
                             math.pi * 
                             math.cos(partOptions['wings'].loc[part]['angle of attack'])) / 
                            ((partOptions['wings'].loc[part]['width'] * 
                              math.cos(partOptions['wings'].loc[part]['angle of attack'])) + 
                             (2 * partOptions['wings'].loc[part]['length']))) * 
                            x['side wing'][part] for part in partOptions['wings'].index.values)

total_downforce = rw_downforce + fw_downforce + (2 * sw_downforce)

m.addConstr(z['downforce'] == total_downforce)
m.update()

In [153]:
# Acceleration Constraint

C = (0.005 + (sum(u_rt_pressure_inverse[part] for part in partOptions['tires'].index.values)) *
            (0.01 + (0.0095 * (((v_car * 3.6) / (100)) ** 2))))

F_wheels = gp.quicksum((partOptions['engine'].loc[part]['Torque'] * 
                       (3600 * 2 * math.pi / 60) / 
                       (v_car)) * 
                       x['engine'][part] for part in partOptions['engine'].index.values)

total_resistance = (gp.quicksum(total_drag * x['engine'][part] for part in partOptions['engine'].index.values) + 
                   gp.quicksum(C * g * u_e_mTotal[part] for part in partOptions['engine'].index.values))
    
m.addConstr(z['acceleration'] * z['mass'] == F_wheels - total_resistance)
m.addConstrs(u_e_mTotal[part] == z['drag'] * x['engine'][part] for part in partOptions['engine'].index.values)
m.addConstrs(u_rt_pressure_inverse[part] * partOptions['tires'].loc[part]['pressure'] * x['rear tire'][part] == x['rear tire'][part] for part in partOptions['tires'].index.values)
m.update()

In [154]:
# Crash Force Constraints

m.addConstr(z['crash force'] == gp.quicksum((math.sqrt((v_car ** 2) * 
                                                       partOptions['impactattenuator'].loc[part]['width'] * 
                                                       partOptions['impactattenuator'].loc[part]['height'] * 
                                                       partOptions['impactattenuator'].loc[part]['E'] / 
                                                       (2 * partOptions['impactattenuator'].loc[part]['length']))) * 
                                            u_rootMass * 
                                            x['impactattenuator'][part] for part in partOptions['impactattenuator'].index.values))
                                                  
m.addConstr(u_rootMass * u_rootMass == z['mass'])
m.update()

In [155]:
# Impact Attenuator Volume Constraints

attenuator_volume = gp.quicksum(partOptions['impactattenuator'].loc[part]['length'] * 
                                partOptions['impactattenuator'].loc[part]['width'] * 
                                partOptions['impactattenuator'].loc[part]['height'] * 
                                x['impactattenuator'][part] for part in partOptions['impactattenuator'].index.values)

m.addConstr(z['attenuator volume'] == attenuator_volume)
m.update()

In [156]:
# Cornering Velocity Constraints

v_cor_forces = gp.quicksum((z['downforce'] + 
                            (z['mass'] * g) - 
                            (2 * (y_parameter * partOptions['suspension'].loc[part]['krsp'] + 
                                  y_dot_parameter * partOptions['suspension'].loc[part]['crsp'])) - 
                            (2 * (y_parameter * partOptions['suspension'].loc[part]['kfsp'] + 
                                  y_dot_parameter * partOptions['suspension'].loc[part]['cfsp']))) * 
                            x['suspension'][part] for part in partOptions['suspension'].index.values)
            
m.addConstr(z['cornering velocity'] * z['mass'] == v_cor_forces * C_lat * r_track)
m.update()

In [157]:
# Braking Distance

F_y = ((z['mass'] * g) + 
       (z['downforce']) - 
       (gp.quicksum(2 * (y_parameter * partOptions['suspension'].loc[part]['krsp'] + 
                         y_dot_parameter * partOptions['suspension'].loc[part]['crsp']) * 
                    x['suspension'][part] for part in partOptions['suspension'].index.values)) - 
       (gp.quicksum(2 * (y_parameter * partOptions['suspension'].loc[part]['kfsp'] + 
                         y_dot_parameter * partOptions['suspension'].loc[part]['cfsp']) * 
                    x['suspension'][part] for part in partOptions['suspension'].index.values)))

lhs_remainder = (gp.quicksum(u_rt_brkDistance[part] * 
                             partOptions['tires'].loc[part]['radius'] * 
                             (0.005 + 
                              (1 / partOptions['tires'].loc[part]['pressure']) * 
                              (0.01 + 0.0095 * (((v_car * 3.6) / (100)) ** 2))) for part in partOptions['tires'].index.values))

lhs = F_y * lhs_remainder

m.addConstr(lhs == ((gp.quicksum((v_car ** 2) * 
                                 (partOptions['tires'].loc[part]['radius'] * 
                                  x['rear tire'][part]) * 
                                (z['mass']) for part in partOptions['tires'].index.values)) - 
                    (gp.quicksum(z['braking distance'] * 
                                 8 * 
                                 c_brk * 
                                 P_brk * 
                                 partOptions['brakes'].loc[part]['hbrk'] * 
                                 partOptions['brakes'].loc[part]['wbrk'] * 
                                 partOptions['brakes'].loc[part]['rbrk'] * 
                                 x['brakes'][part] for part in partOptions['brakes'].index.values))))

m.addConstrs(u_rt_brkDistance[part] == z['braking distance'] * x['rear tire'][part] for part in partOptions['tires'].index.values)
m.update()

In [158]:
# Suspension Acceleration Constraints

m.addConstr(z['suspension acceleration'] * z['mass'] == (gp.quicksum(((-2 * (y_parameter * partOptions['suspension'].loc[part]['kfsp'] + 
                                                                            y_dot_parameter * partOptions['suspension'].loc[part]['cfsp'])) + 
                                                                     (2 * (y_parameter * partOptions['suspension'].loc[part]['krsp'] + 
                                                                           y_dot_parameter * partOptions['suspension'].loc[part]['crsp'])) + 
                                                                     (z['mass'] * g) + 
                                                                     (z['downforce'])) * 
                                                                    x['suspension'][part] for part in partOptions['suspension'].index.values)))
m.update()

In [159]:
# Pitch Moment Constraints

sp_pitch_moment = gp.quicksum(((2 * (y_parameter * partOptions['suspension'].loc[part]['kfsp'] + 
                                    y_dot_parameter * partOptions['suspension'].loc[part]['cfsp'])) + 
                              (2 * (y_parameter * partOptions['suspension'].loc[part]['krsp'] + 
                                    y_parameter * partOptions['suspension'].loc[part]['crsp']))) * 
                             x['suspension'][part] for part in partOptions['suspension'].index.values)

rw_pitch_moment = gp.quicksum(rw_downforce * 
                              (partOptions['cabin'].loc[cabin]['length'] - partOptions['wings'].loc[wing]['length']) * 
                              u_rw_c[wing, cabin] for wing in partOptions['wings'].index.values for cabin in partOptions['cabin'].index.values)

fw_pitch_moment = gp.quicksum(fw_downforce * 
                              (partOptions['cabin'].loc[cabin]['length'] - partOptions['wings'].loc[wing]['length']) * 
                              u_fw_c[wing, cabin] for wing in partOptions['wings'].index.values for cabin in partOptions['cabin'].index.values)

sw_pitch_moment = gp.quicksum(sw_downforce * 
                              (partOptions['cabin'].loc[cabin]['length'] - partOptions['wings'].loc[wing]['length']) * 
                              u_fw_c[wing, cabin] for wing in partOptions['wings'].index.values for cabin in partOptions['cabin'].index.values)

pitch_moment_total = sp_pitch_moment + rw_pitch_moment - fw_pitch_moment - (2 * sw_pitch_moment)

m.addConstrs(u_rw_c[wing, cabin] == x['rear wing'][wing] * x['cabin'][cabin] for wing in partOptions['wings'].index.values for cabin in partOptions['cabin'].index.values)
m.addConstrs(u_fw_c[wing, cabin] == x['front wing'][wing] * x['cabin'][cabin] for wing in partOptions['wings'].index.values for cabin in partOptions['cabin'].index.values)
m.addConstrs(u_sw_c[wing, cabin] == x['side wing'][wing] * x['cabin'][cabin] for wing in partOptions['wings'].index.values for cabin in partOptions['cabin'].index.values)

m.addConstr(z['pitch moment'] == pitch_moment_total)
m.update()

# Set model non-convex parameter = 2
m.params.NonConvex = 2

Set parameter NonConvex to value 2


In [160]:
# Constraints on z_norm variables

for obj in objectives:
    if obj != 'downforce' and obj != 'acceleration' and obj != 'cornering velocity':
        m.addConstr((z_norm[obj] == (z[obj] - objectives_min[obj]) / 
                                    (objectives_max[obj] - objectives_min[obj])), name = 'min objectives normalization')
    else:
        m.addConstr((z_norm[obj] == (objectives_max[obj] - z[obj]) / 
                                    (objectives_max[obj] - objectives_min[obj])), name = 'max objectives normalization')

In [161]:
# Constraints to enforce Z_star Solution

m.addConstr(x['rear wing'][1143] == 1)
m.addConstr(x['front wing'][1642] == 1)
m.addConstr(x['side wing'][1143] == 1)
m.addConstr(x['rear tire'][28] == 1)
m.addConstr(x['front tire'][28] == 1)
m.addConstr(x['engine'][20] == 1)
m.addConstr(x['cabin'][468] == 1)
m.addConstr(x['impactattenuator'][8] == 1)
m.addConstr(x['brakes'][32] == 1)
m.addConstr(x['suspension'][1] == 1)
m.update()

In [162]:
#m.write("model.lp")

## Optimize, View Output, and Get Status


In [163]:
m.optimize()

saedfsc.handleStatus(m)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[rosetta2] - Darwin 22.4.0 22E252)

CPU model: Apple M1 Max
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 365 rows, 2168 columns and 2656 nonzeros
Model fingerprint: 0x8c68bc84
Model has 56 quadratic objective terms
Model has 1365 quadratic constraints
Variable types: 178 continuous, 1990 integer (1920 binary)
Coefficient statistics:
  Matrix range     [1e-09, 8e+04]
  QMatrix range    [5e-03, 1e+07]
  QLMatrix range   [2e-04, 8e+04]
  Objective range  [1e+05, 1e+05]
  QObjective range [1e+00, 1e+01]
  Bounds range     [4e-03, 7e+08]
  RHS range        [3e-06, 1e+00]
  QRHS range       [9e-02, 4e-01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 305 rows and 2072 columns
Presolve time: 0.01s
Presolved: 239 rows, 138 columns, 602 nonzeros
Presolved model has 4 quadratic constraint(s)
Presolved model 

## Get profit and revenue.

In [164]:
print("Profit: $", format(m.ObjVal, ",.2f"))
print("Revenue: $", format(price*meanDemandVar.X, ",.2f"))

Profit: $ 589,805.23
Revenue: $ 1,661,067.15


## Create Output DataFrames

In [241]:
sInVals = [round(sIn[i].X) for i in  G.nodes()]
sOutVals = [round(sOut[i].X) for i in  G.nodes()]
netTimeVals = [round(netTime[i].X) for i in  G.nodes()]
holdingCostVals = [holdCostCoef*(stdDevDemandTimesCumCostVar[i].X)*sqrtNetTime[i].X for i in  G.nodes()]
pipelineCostVals = [holdingCostRate*(cumCostVar[i].X - stageCost[i].X/2)*stageTime[i].X*meanDemandVar.X 
                    for i in  G.nodes()]
cogsVals = [betaMult*stageCost[i].X*meanDemandVar.X for i in  G.nodes()]

df1a = pd.DataFrame({
    'SC Node': G.nodes(),
    'Proc. time': [stageTime[i].X for i in  G.nodes()],
    'Stage cost': [stageCost[i].X for i in  G.nodes()],
    'Cum. cost': [cumCostVar[i].X for i in  G.nodes()],
    'sIn': sInVals,
    'sOut': sOutVals,
    'Max time' : [maxServiceTimeOut[i] for i in  G.nodes()],
    'netTime': netTimeVals,
    'holding cost' : holdingCostVals,
    'pipeline cost' : pipelineCostVals,
    'COGS' : cogsVals
})

df1b1 = pd.DataFrame({
    'SC Node': G.nodes(),
    'Proc. time': [round(stageTime[i].X) for i in  G.nodes()],
    'Stage cost': [round(stageCost[i].X) for i in  G.nodes()],
    #'Cum. cost': [round(cumCostVar[i].X) for i in  G.nodes()]
})

df1b2 = pd.DataFrame({
    'SC Node': G.nodes(),
    'sIn': sInVals,
    'sOut': sOutVals,
    #'Max time' : [maxServiceTimeOut[i] for i in  G.nodes()],
    'Net': netTimeVals,
    'Holding' : holdingCostVals,
    'Pipeline' : pipelineCostVals,
    'COGS' : cogsVals
})

df2 = pd.DataFrame({
    'Arc': G.edges(),
    'Selected': [arcSelect[i,j].X for i,j in  G.edges()]
})

df3 = pd.DataFrame({
    'Component': componentSelect.keys(),
    'Selected': [componentSelect[i].X for i in componentSelect.keys()]
})

df4 = pd.DataFrame({
    'Customer': customerTypes,
    'Qty': [meanDemandForCustomerType[c] for c in customerTypes],
    'Total Utility for Competitors': [getTotalUtilityForCompetitors(c) for c in customerTypes],
    'Utility for Product' : [sum([cWtsPerf[c][o]*z_norm[objectives[o]].X for o in objIndices]) 
                             for c in customerTypes],
    'Market Share': [marketShare[c].X for c in customerTypes]
})

df4b = pd.DataFrame({
    'Customer Type': range(1, len(customerTypes)+1),
    'Total Utility for Competitors': [round(getTotalUtilityForCompetitors(c),1) for c in customerTypes],
    'Utility for Product' : [round(sum([cWtsPerf[c][o]*z_norm[objectives[o]].X for o in objIndices]), 2) 
                             for c in customerTypes],
    'Market Share': [round(marketShare[c].X,2) for c in customerTypes]
})

df5 = pd.DataFrame({
    'Objective': objIndices,
    'Value': [z[o].X for o in objectives]
})

## Inventory DataFrame

In [230]:
df1a

,SC Node,Proc. time,Stage cost,Cum. cost,sIn,sOut,Max time,netTime,holding cost,pipeline cost,COGS
0,rear wing,6.0,47.0,47.0,0,0,8,6,483.604142,2342.104680,3903.507801
1,front wing,14.0,370.0,370.0,0,4,4,10,4914.940364,43021.639164,30729.742260
2,side wing,6.0,47.0,47.0,0,5,8,1,197.430564,2342.104680,3903.507801
3,front tire,8.0,336.0,336.0,0,6,16,2,1996.047186,22324.742485,27905.928106
4,rear tire,8.0,319.0,319.0,0,0,4,8,3790.113408,21195.216824,26494.021030
5,engine,6.0,1649.0,1649.0,0,5,10,1,6926.872346,82172.991870,136954.986450
6,cabin,10.0,140.0,140.0,0,5,8,5,1315.011547,11627.470044,11627.470044
7,impactattenuator,13.0,35.0,35.0,0,6,16,7,388.985661,3778.927764,2906.867511
8,brakes,10.0,0.0,0.0,0,0,2,10,0.000000,0.000000,0.000000
9,suspension,14.0,0.0,0.0,0,6,10,8,0.000000,0.000000,0.000000


In [231]:
latex_table = df1b1.to_latex(index=False)
print(latex_table)

\begin{tabular}{lrr}
\toprule
SC Node & Proc. time & Stage cost \\
\midrule
rear wing & 6 & 47 \\
front wing & 14 & 370 \\
side wing & 6 & 47 \\
front tire & 8 & 336 \\
rear tire & 8 & 319 \\
engine & 6 & 1649 \\
cabin & 10 & 140 \\
impactattenuator & 13 & 35 \\
brakes & 10 & 0 \\
suspension & 14 & 0 \\
midbody & 7 & 8 \\
front & 6 & 8 \\
rear & 5 & 2 \\
FINAL & 6 & 9 \\
\bottomrule
\end{tabular}



In [232]:
df1b2['Holding'] = df1b2['Holding'].astype(int)
df1b2['Pipeline'] = df1b2['Pipeline'].astype(int)
df1b2['COGS'] = df1b2['COGS'].astype(int)
latex_table = df1b2.to_latex(index=False)
print(latex_table)

\begin{tabular}{lrrrrrr}
\toprule
SC Node & sIn & sOut & Net & Holding & Pipeline & COGS \\
\midrule
rear wing & 0 & 0 & 6 & 483 & 2342 & 3903 \\
front wing & 0 & 4 & 10 & 4914 & 43021 & 30729 \\
side wing & 0 & 5 & 1 & 197 & 2342 & 3903 \\
front tire & 0 & 6 & 2 & 1996 & 22324 & 27905 \\
rear tire & 0 & 0 & 8 & 3790 & 21195 & 26494 \\
engine & 0 & 5 & 1 & 6926 & 82172 & 136954 \\
cabin & 0 & 5 & 5 & 1315 & 11627 & 11627 \\
impactattenuator & 0 & 6 & 7 & 388 & 3778 & 2906 \\
brakes & 0 & 0 & 10 & 0 & 0 & 0 \\
suspension & 0 & 6 & 8 & 0 & 0 & 0 \\
midbody & 5 & 12 & 0 & 0 & 213945 & 664 \\
front & 6 & 12 & 0 & 0 & 74249 & 664 \\
rear & 0 & 4 & 1 & 1545 & 30480 & 166 \\
FINAL & 12 & 18 & 0 & 0 & 295553 & 747 \\
\bottomrule
\end{tabular}



In [234]:
df1b2.loc['Total', ['Holding', 'Pipeline', 'COGS']] = df1b2[['Holding', 'Pipeline', 'COGS']].sum()
df1b2['Holding'] = df1b2['Holding'].astype(int)
df1b2['Pipeline'] = df1b2['Pipeline'].astype(int)
df1b2['COGS'] = df1b2['COGS'].astype(int)
latex_table = df1b2.to_latex(index=False)
print(latex_table)

\begin{tabular}{lrrrrrr}
\toprule
SC Node & sIn & sOut & Net & Holding & Pipeline & COGS \\
\midrule
rear wing & 0.000000 & 0.000000 & 6.000000 & 483 & 2342 & 3903 \\
front wing & 0.000000 & 4.000000 & 10.000000 & 4914 & 43021 & 30729 \\
side wing & 0.000000 & 5.000000 & 1.000000 & 197 & 2342 & 3903 \\
front tire & 0.000000 & 6.000000 & 2.000000 & 1996 & 22324 & 27905 \\
rear tire & 0.000000 & 0.000000 & 8.000000 & 3790 & 21195 & 26494 \\
engine & 0.000000 & 5.000000 & 1.000000 & 6926 & 82172 & 136954 \\
cabin & 0.000000 & 5.000000 & 5.000000 & 1315 & 11627 & 11627 \\
impactattenuator & 0.000000 & 6.000000 & 7.000000 & 388 & 3778 & 2906 \\
brakes & 0.000000 & 0.000000 & 10.000000 & 0 & 0 & 0 \\
suspension & 0.000000 & 6.000000 & 8.000000 & 0 & 0 & 0 \\
midbody & 5.000000 & 12.000000 & 0.000000 & 0 & 213945 & 664 \\
front & 6.000000 & 12.000000 & 0.000000 & 0 & 74249 & 664 \\
rear & 0.000000 & 4.000000 & 1.000000 & 1545 & 30480 & 166 \\
FINAL & 12.000000 & 18.000000 & 0.000000 & 0 & 295

## Supply Chain Network Design DataFrame

In [168]:
print(df2)
print(df3)

                          Arc  Selected
0           (rear wing, rear)       1.0
1         (front wing, front)       1.0
2        (side wing, midbody)       1.0
3         (front tire, front)       1.0
4           (rear tire, rear)       1.0
5           (engine, midbody)       1.0
6            (cabin, midbody)       1.0
7   (impactattenuator, front)       1.0
8              (brakes, rear)       1.0
9         (suspension, front)       1.0
10           (midbody, FINAL)       1.0
11             (front, FINAL)       1.0
12              (rear, FINAL)       1.0
          Component  Selected
0         rear wing       1.0
1        front wing       1.0
2         side wing       1.0
3        front tire       1.0
4         rear tire       1.0
5            engine       1.0
6             cabin       1.0
7  impactattenuator       1.0
8            brakes       1.0
9        suspension       1.0


## Objectives, Utility, and Demand DataFrame

In [169]:
print(df5)
df4

    Objective         Value
0           0  3.247361e+02
1           1  2.689159e-01
2           2  8.242691e+01
3           3  1.551951e+03
4           4  2.000000e-01
5           5  1.145411e+07
6           6  5.000000e-02
7           7  1.497094e+01
8           8  3.435307e+01
9           9  1.458911e+01
10         10  1.023978e+03


,Customer,Qty,Total Utility for Competitors,Utility for Product,Market Share
0,CustomerType1,8,1.143355,0.250821,0.322761
1,CustomerType2,16,0.737972,0.201459,0.466677
2,CustomerType3,16,0.907010,0.299588,0.410110


In [245]:
pd.options.display.float_format = "{:.2f}".format
df4b['Total Utility for Competitors'] = df4b['Total Utility for Competitors'].round(2)
# df1b2['Pipeline'] = df1b2['Pipeline'].astype(int)
# df1b2['COGS'] = df1b2['COGS'].astype(int)
latex_table = df4b.to_latex(index=False)
print(latex_table)

\begin{tabular}{rrrr}
\toprule
Customer Type & Total Utility for Competitors & Utility for Product & Market Share \\
\midrule
1 & 1.100000 & 0.250000 & 0.320000 \\
2 & 0.700000 & 0.200000 & 0.470000 \\
3 & 0.900000 & 0.300000 & 0.410000 \\
\bottomrule
\end{tabular}



In [170]:
# Product Design Optimization Problem Solution

for part in partOptions['wings'].index.values:
    if x['rear wing'][part].X >= 0.5:
        print('Rear Wing', part, 'chosen')
        
for part in partOptions['wings'].index.values:
    if x['front wing'][part].X >= 0.5:
        print('Front Wing', part, 'chosen')
        
for part in partOptions['wings'].index.values:
    if x['side wing'][part].X >= 0.5:
        print('Side Wing', part, 'chosen')
        
for part in partOptions['tires'].index.values:
    if x['rear tire'][part].X >= 0.5:
        print('Rear Tire', part, 'chosen')
        
for part in partOptions['tires'].index.values:
    if x['front tire'][part].X >= 0.5:
        print('Front Tire', part, 'chosen')
        
for part in partOptions['engine'].index.values:
    if x['engine'][part].X >= 0.5:
        print('Engine', part, 'chosen')
        
for part in partOptions['cabin'].index.values:
    if x['cabin'][part].X >= 0.5:
        print('Cabin', part, 'chosen')
        
for part in partOptions['impactattenuator'].index.values:
    if x['impactattenuator'][part].X >= 0.5:
        print('Attenuator', part, 'chosen')
        
for part in partOptions['brakes'].index.values:
    if x['brakes'][part].X >= 0.5:
        print('Brake', part, 'chosen')
        
for part in partOptions['suspension'].index.values:
    if x['suspension'][part].X >= 0.5:
        print('Suspension', part, 'chosen')
        
print()
print('Mass Objective:', z['mass'].X)
print('CoG Objective:', z['center of gravity'].X)
print('Drag Objective:', z['drag'].X)
print('Downforce Objective:', z['downforce'].X)
print('Acceleration Objective:', z['acceleration'].X)
print('Crash Force Objective:', z['crash force'].X)
print('Impact Attenuator Volume Objective:', z['attenuator volume'].X)
print('Cornering Velocity Objective:', z['cornering velocity'].X)
print('Braking Distance Objective:', z['braking distance'].X)
print('Suspension Acceleration Objective:', z['suspension acceleration'].X)
print('Pitch Moment Objective:', z['pitch moment'].X)

Rear Wing 1143 chosen
Front Wing 1642 chosen
Side Wing 1143 chosen
Rear Tire 28 chosen
Front Tire 28 chosen
Engine 20 chosen
Cabin 468 chosen
Attenuator 8 chosen
Brake 32 chosen
Suspension 1 chosen

Mass Objective: 324.73606496
CoG Objective: 0.26891588071304406
Drag Objective: 82.42690704033977
Downforce Objective: 1551.950512705599
Acceleration Objective: 0.2
Crash Force Objective: 11454109.201700917
Impact Attenuator Volume Objective: 0.05
Cornering Velocity Objective: 14.970936055620786
Braking Distance Objective: 34.353068755598294
Suspension Acceleration Objective: 14.589113502212216
Pitch Moment Objective: 1023.9783968698075


## Display Model
Uncomment this code to display the model.

In [171]:
#!cat model.lp